<a href="https://colab.research.google.com/github/HuyenNguyenHelen/ABSA-BERT-pair/blob/master/EDA_topic_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Loading the data

In [3]:
with open(r'/content/158_comments.csv', 'r', encoding = 'utf-8') as f:
  data = pd.read_csv(f)
data

,docketId,documentType,govAgency,firstName,lastName,organization,Comment
0,ATF-2021-0001,Public Submission,NaN,John,Wessinger,NaN,ATF 2021R-05 is a huge overstep into the right...
1,ATF-2021-0001,Public Submission,NaN,Michael,Dyer,NaN,RE: ATF 2021R-05<br/><br/>ATF&#39;s efforts to...
2,ATF-2021-0001,Public Submission,NaN,Dustin,Jose,NaN,I don&#39;t believe the ATF is doing it&#39;s ...
3,ATF-2021-0001,Public Submission,NaN,Anonymous,Anonymous,NaN,Point 1: The ATF should not be proposing or ma...
4,ATF-2021-0001,Public Submission,NaN,Robert,Caddell Jr,NaN,I have also yet to see a news story involving ...
...,...,...,...,...,...,...,...
152,ATF-2021-0001,Public Submission,NaN,Steve,Ray,NaN,RE: ATF 2021R-05<br/><br/>It has not escaped a...
153,ATF-2021-0001,Public Submission,NaN,Mat,Hawk,NaN,RE: ATF 2021R-05<br/><br/>It has not escaped a...
154,ATF-2021-0001,Public Submission,NaN,Rodney,Landry,NaN,RE: ATF 2021R-05<br/><br/>Since the founding o...
155,ATF-2021-0001,Public Submission,NaN,Jay,Raulerson,NaN,I am a Police Officer and have been for 15+ ye...


# EDA

In [9]:
print(data['Comment'][78])

With its definitions of \&quot;partially complete receiver\&quot; and \&quot;readily,\&quot; ATF 2021R-05 would codify arbitrary and capricious classification standards. The vague definition of \&quot;partially complete receiver\&quot; contains still more vague definitions, like \&quot;clearly identifiable\&quot;, \&quot;unfinished component part of a weapon\&quot;, \&quot;critical stage of manufacture\&quot;, \&quot;sufficiently complete to function\&quot;, and \&quot;primordial state.\&quot;  The new definition of \&quot;readily\&quot; contains so many factors, and of such ambiguous weight and meaning, that the test can only be meant to confuse and intimidate non-FFL entities before ATF\&#39;s awesome authority. The multi-factor test for \&quot;readily\&quot; amounts to lawless Caesarism, and invests the ATF Director with more power than Congress has authorized. ATF would like to legitimize its wholly arbitrary sample submission process-- one which is not derived from the GCA, and wh

In [10]:
def number_para (text):
  n_para = len(text.split('<br/>'))
  return n_para

def number_words (text):
  n_words = len(text.split())
  return n_words

df_n_para = data['Comment'].apply(lambda x: number_para (x) )
df_n_words = data['Comment'].apply(lambda x: number_words (x) )
df_n_para.sort_values(ascending = False)


73     30
18     23
81     22
78     22
42     21
       ..
106     1
105     1
62      1
102     1
68      1
Name: Comment, Length: 157, dtype: int64

In [5]:
df_n_words.sort_values(ascending = False)

78     709
81     709
18     707
38     541
42     480
      ... 
26      17
102     15
91      11
105     11
9        6
Name: Comment, Length: 157, dtype: int64

In [13]:
for i in data['Comment']:
  if len(i.split('<br/>')) > 0:
      for p in i.split('<br/>'):
        print('-------------     ', p)

-------------      ATF 2021R-05 is a huge overstep into the rights of American Citizens.  The rule changes proposed will in no way limit crimes committed.  In truth, this law will only make law abiding citizens criminals due to the rule changes.  Criminals do not care if they break the law.  More rules will not change their minds.
-------------      
-------------      This proposal is in direct violation of the US Constitution in Article 1 Section 9: No Bill of Attainder or ex po facto Law shall be passed.  When owning certain items is legal up until the day ATF 2021R-05 goes into law and immediately afterwards, those items are no longer legal, this becomes an ex po facto law.  This cannot occur.  Even with the Assault Weapons Ban items that were already owned were grandfathered in and no one was made into a criminal.
-------------      
-------------      ATF 2021R-05 does not need to be passed as it is in violation of the law of the land.  It violates Article 1 Section 9.  It violat

# Topic extraction

In [30]:
!pip install stop-words
from stop_words import get_stop_words
stopwords = get_stop_words('en')
from textblob import Word
import re
import nltk
nltk.download('wordnet')
def utils_preprocess_text(txt, lst_regex=None, punkt=True, lower=True, lst_stopwords=None, stemm=False, lemm=True):
    ## regex (in case, before processing)
    if lst_regex is not None: 
        for regex in lst_regex:
            txt = re.sub(regex, '', txt)
    
    ## clean 
    ### separate sentences with '. '
    txt = re.sub(r'\.(?=[^ \W\d])', '. ', str(txt))
    ### remove punctuations and characters
    txt = re.sub(r'[^\w\s]', '', txt) if punkt is True else txt
    ### strip
    txt = " ".join([word.strip() for word in txt.split()])
    ### lowercase
    txt = txt.lower() if lower is True else txt
            
    ## Tokenize (convert from string to list)
    lst_txt = txt.split()

    ## remove Stopwords
    if lst_stopwords is not None:
        lst_txt = [word for word in lst_txt if word not in lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if stemm is True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_txt = [ps.stem(word) for word in lst_txt]
                
    ## Lemmatization (convert the word into root word)
    if lemm is True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_txt = [lem.lemmatize(word) for word in lst_txt]

    ## remove leftover Stopwords
    if lst_stopwords is not None:
        lst_txt = [word for word in lst_txt if word not in lst_stopwords]
            
    ## back to string from list
    txt = " ".join(lst_txt)
    return txt

data['clean_sent'] = data['Comment'].apply( lambda x:utils_preprocess_text(x, stemm = False, lemm = True, lst_stopwords=stopwords))
df.head()

  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32912 sha256=6391786bf151d42ab3923fec7e02034e7422e6c27e54805bac42d0c8763e2788
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


,docketId,documentType,govAgency,firstName,lastName,organization,Comment,Topic1,Topic2,Topic3,Topic4,Topic5,Dominant_topic
0,ATF-2021-0001,Public Submission,NaN,John,Wessinger,NaN,ATF 2021R-05 is a huge overstep into the right...,1.0,0.00,0.0,0.00,0.00,1
1,ATF-2021-0001,Public Submission,NaN,Michael,Dyer,NaN,RE: ATF 2021R-05<br/><br/>ATF&#39;s efforts to...,0.0,0.88,0.0,0.00,0.11,2
2,ATF-2021-0001,Public Submission,NaN,Dustin,Jose,NaN,I don&#39;t believe the ATF is doing it&#39;s ...,0.0,0.45,0.0,0.54,0.00,4
3,ATF-2021-0001,Public Submission,NaN,Anonymous,Anonymous,NaN,Point 1: The ATF should not be proposing or ma...,0.0,1.00,0.0,0.00,0.00,2
4,ATF-2021-0001,Public Submission,NaN,Robert,Caddell Jr,NaN,I have also yet to see a news story involving ...,0.0,0.00,0.0,1.00,0.00,4


In [31]:
# initialise the count vectorizer

vectorizer = CountVectorizer(analyzer = 'word', ngram_range = (1, 2))
                            
# join the processed data to be vectorised

vectors = []

for row in data['clean_sent']:
    vectors.append(", ".join(row.split()))

vectorised = vectorizer.fit_transform(vectors)

print(vectorised)


# initisalise LDA Model

lda_model = LatentDirichletAllocation(n_components = 5, # number of topics
                                  random_state = 10,          # random state
                                  evaluate_every = -1,      # compute perplexity every n iters, default: Don't
                                  n_jobs = -1,              # Use all available CPUs
                                 )

lda_output = lda_model.fit_transform(vectorised)

# column names

topic_names = ["Topic" + str(i) for i in range(1, lda_model.n_components + 1)]

# make the pandas dataframe

df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns = topic_names)

# get dominant topic for each document

dominant_topic = (np.argmax(df_document_topic.values, axis=1)+1)
df_document_topic['Dominant_topic'] = dominant_topic

# join to original dataframes

df = pd.merge(data, df_document_topic, left_index = True, right_index = True, how = 'outer')
display(df.head(10))

  (0, 858)	2
  (0, 49)	3
  (0, 4285)	1
  (0, 6073)	1
  (0, 7530)	2
  (0, 593)	1
  (0, 1788)	3
  (0, 7610)	3
  (0, 1676)	3
  (0, 6702)	1
  (0, 9181)	3
  (0, 9080)	1
  (0, 5181)	1
  (0, 2388)	1
  (0, 1989)	1
  (0, 8683)	1
  (0, 4859)	7
  (0, 5319)	1
  (0, 155)	1
  (0, 2426)	2
  (0, 2956)	1
  (0, 1609)	1
  (0, 1352)	1
  (0, 5561)	1
  (0, 6679)	1
  :	:
  (156, 9302)	1
  (156, 7173)	1
  (156, 2208)	1
  (156, 7581)	1
  (156, 8153)	1
  (156, 1800)	1
  (156, 2359)	1
  (156, 3587)	1
  (156, 4776)	1
  (156, 7712)	1
  (156, 1275)	1
  (156, 1404)	1
  (156, 6709)	1
  (156, 2258)	1
  (156, 4972)	1
  (156, 6718)	1
  (156, 7325)	1
  (156, 6930)	1
  (156, 4116)	1
  (156, 2427)	1
  (156, 183)	1
  (156, 5899)	1
  (156, 8894)	1
  (156, 3550)	1
  (156, 1986)	1


,docketId,documentType,govAgency,firstName,lastName,organization,Comment,clean_sent,Topic1,Topic2,Topic3,Topic4,Topic5,Dominant_topic
0,ATF-2021-0001,Public Submission,NaN,John,Wessinger,NaN,ATF 2021R-05 is a huge overstep into the right...,atf 2021r05 huge overstep right american citiz...,0.00,1.00,0.00,0.00,0.00,2
1,ATF-2021-0001,Public Submission,NaN,Michael,Dyer,NaN,RE: ATF 2021R-05<br/><br/>ATF&#39;s efforts to...,re atf 2021r05brbratf39s effort turn leftwing ...,1.00,0.00,0.00,0.00,0.00,1
2,ATF-2021-0001,Public Submission,NaN,Dustin,Jose,NaN,I don&#39;t believe the ATF is doing it&#39;s ...,don39t believe atf it39s job defending right c...,0.98,0.01,0.01,0.01,0.01,1
3,ATF-2021-0001,Public Submission,NaN,Anonymous,Anonymous,NaN,Point 1: The ATF should not be proposing or ma...,point 1 atf proposing making law left legislat...,0.99,0.00,0.00,0.00,0.00,1
4,ATF-2021-0001,Public Submission,NaN,Robert,Caddell Jr,NaN,I have also yet to see a news story involving ...,also yet see news story involving use one gun ...,0.00,0.00,1.00,0.00,0.00,3
5,ATF-2021-0001,Public Submission,NaN,Ryan,Miller,NaN,"This complicates what parts are, was and will ...",complicates part will firearm make gun part tr...,0.00,0.00,0.00,0.00,1.00,5
6,ATF-2021-0001,Public Submission,NaN,David,Sowers,NaN,These raw changes would have no effect on crim...,raw change effect crime just put undue hardshi...,0.00,0.00,0.00,0.99,0.00,4
7,ATF-2021-0001,Public Submission,NaN,Neil,Baker,NaN,My understanding of how our government works i...,understanding government work congress senate ...,0.00,0.99,0.00,0.00,0.00,2
8,ATF-2021-0001,Public Submission,NaN,CJ,Green,NaN,RE: ATF 2021R-05<br/><br/>It has not escaped a...,re atf 2021r05brbrit escaped anyone39s attenti...,0.00,0.00,1.00,0.00,0.00,3
9,ATF-2021-0001,Public Submission,NaN,Meredith,OMalley,NaN,Ghost guns kill! Ban ghost guns,ghost gun kill ban ghost gun,0.02,0.93,0.02,0.02,0.02,2


In [32]:
# index names
docnames = ['Doc' + str(i) for i in range(len(data))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topic_names, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(lda_model.components_)

# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topic_names

df_topic_no = pd.DataFrame(df_topic_keywords.idxmax())
df_scores = pd.DataFrame(df_topic_keywords.max())

tmp = pd.merge(df_topic_no, df_scores, left_index=True, right_index=True)
tmp.columns = ['topic', 'relevance_score']

display(tmp)

,topic,relevance_score
100,Topic1,6.200000
100 page,Topic1,6.200000
101,Topic1,3.200000
101 gca,Topic1,3.200000
10th,Topic1,1.199995
...,...,...
yr stop,Topic3,1.199999
zero,Topic3,2.199953
zero affect,Topic3,1.199998
zero chance,Topic5,1.200000


In [33]:

all_topics = []

for i in tmp['topic'].unique():    
    tmp_1 = tmp.loc[tmp['topic'] == i].reset_index()
    tmp_1 = tmp_1.sort_values('relevance_score', ascending=False).head(1)

    # tmp_1['topic'] = tmp_1['topic'] + 1
    
    tmp_2 = []
    tmp_2.append(tmp_1['topic'].unique()[0])
    tmp_2.append(list(tmp_1['index'].unique()))
    all_topics.append(tmp_2)

all_topics = pd.DataFrame(all_topics, columns=['Dominant_topic', 'topic_name'])
display(all_topics)

,Dominant_topic,topic_name
0,Topic1,[atf]
1,Topic4,[law]
2,Topic5,[gun owner]
3,Topic3,[br]
4,Topic2,[proposal]
